In [2]:
from fredapi import Fred
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import plotly.graph_objects as go
import datetime
import pandas as pd
import csv
import os
import time
from dateutil import tz
import glob
import quandl as q
import re
import plotly.graph_objects as go
import numpy as np

api_key_fred = os.environ['api_key_fred']
fred = Fred(api_key=api_key_fred)

In [3]:


quandl_api_key = os.environ['quandl_api_key']

# # # start - read in BTC data # # #
datasource_btcusd = "BITFINEX/BTCUSD.csv"
btcusd_data = pd.read_csv("coindata/{}".format(
    datasource_btcusd.replace("/", " ")), index_col=0)
btcusd_data.index = pd.to_datetime(btcusd_data.index)

most_recent_stored_btcusd_date = btcusd_data.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")
todays_date = datetime.date.today() - timedelta(days=1)
todays_date = todays_date.strftime("%Y-%m-%d")


if most_recent_stored_btcusd_date != todays_date:
    # start - get data from quandl api
    data = q.get(datasource_btcusd.split(".")[0],   start_date=most_recent_stored_btcusd_date,
                    end_date='{}'.format(todays_date),
                    api_key=quandl_api_key)
    data.info()
    data["First"] = data.Last.shift(1)
    data.dropna()
    btcusd_data = pd.concat([btcusd_data, data])
    btcusd_data = btcusd_data.sort_index()
    # store current df with up-to-date values
    btcusd_data.to_csv('coindata/{}'.format(
        datasource_btcusd.replace("/", " ")), index=True)
        
# # # end - read in BTC data # # #

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 186 entries, 2022-12-16 to 2023-06-19
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   High    186 non-null    float64
 1   Low     186 non-null    float64
 2   Mid     186 non-null    float64
 3   Last    186 non-null    float64
 4   Bid     186 non-null    float64
 5   Ask     186 non-null    float64
 6   Volume  186 non-null    float64
dtypes: float64(7)
memory usage: 11.6 KB


In [4]:
fed_assets_quandl_key = "FED/RESPPA_N_WW"
fed_assets_data = q.get(fed_assets_quandl_key, 
            api_key=quandl_api_key
            )
fed_assets_data = fed_assets_data.dropna()
fed_assets_data.info()
fed_assets_data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1070 entries, 2002-12-18 to 2023-06-14
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Value   1070 non-null   float64
dtypes: float64(1)
memory usage: 16.7 KB


,Value
Date,
2002-12-18,720761.0
2002-12-25,733136.0
2003-01-01,732202.0
2003-01-08,724902.0
2003-01-15,721325.0
...,...
2023-05-17,8456760.0
2023-05-24,8436255.0
2023-05-31,8385854.0


In [5]:
fed_assets_data = fed_assets_data.sort_index()
fed_assets_data

,Value
Date,
2002-12-18,720761.0
2002-12-25,733136.0
2003-01-01,732202.0
2003-01-08,724902.0
2003-01-15,721325.0
...,...
2023-05-17,8456760.0
2023-05-24,8436255.0
2023-05-31,8385854.0


In [6]:
# store current df with up-to-date values
fed_assets_data.to_csv('coindata/{}'.format(
    fed_assets_quandl_key.replace("/", " ")), index=True)

In [7]:
# merge fed and btc
btcusd_data_and_fed = pd.merge(btcusd_data, fed_assets_data, left_index=True, right_index=True, how='left')
btcusd_data_and_fed

,High,Low,Mid,Last,Bid,Ask,Volume,First,Value
Date,,,,,,,,,
2014-04-15,513.9000,452.00,504.23500,505.0000,503.5000,504.97,21013.584774,NaN,NaN
2014-04-16,547.0000,495.00,537.50000,538.0000,537.0000,538.00,29633.358705,505.00,4283967.0
2014-04-17,538.5000,486.10,507.02000,508.0000,506.0400,508.00,20709.783819,538.00,NaN
2014-04-18,509.0000,474.25,483.77000,482.7500,482.7500,484.79,10458.045243,508.00,NaN
2014-04-19,513.9899,473.83,505.01065,507.4999,502.5313,507.49,8963.618369,482.75,NaN
...,...,...,...,...,...,...,...,...,...
2023-06-15,25766.0000,24825.00,25658.50000,25658.0000,25658.0000,25659.00,1128.395333,25035.00,NaN
2023-06-16,26570.0000,25211.00,26380.50000,26374.0000,26380.0000,26381.00,1021.936863,25658.00,NaN
2023-06-17,26857.0000,26202.00,26578.50000,26579.0000,26578.0000,26579.00,784.206533,26374.00,NaN


In [8]:
btcusd_data_and_fed["Value"] = btcusd_data_and_fed["Value"].ffill()
btcusd_data_and_fed = btcusd_data_and_fed.dropna()

In [9]:
btcusd_data_and_fed["BTC_per_FedAssets"] = (btcusd_data_and_fed["Last"] / btcusd_data_and_fed["Value"]) *1000
btcusd_data_and_fed["350_movingaverage_per_FedAssets"] = pd.Series.rolling(btcusd_data_and_fed["BTC_per_FedAssets"], window=350, min_periods=1).mean()
btcusd_data_and_fed["111_movingaverage_per_FedAssets"] = pd.Series.rolling(btcusd_data_and_fed["BTC_per_FedAssets"], window=111, min_periods=1).mean()

C:\Users\gebel\AppData\Local\Temp\ipykernel_29912\2281456468.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btcusd_data_and_fed["BTC_per_FedAssets"] = (btcusd_data_and_fed["Last"] / btcusd_data_and_fed["Value"]) *1000
C:\Users\gebel\AppData\Local\Temp\ipykernel_29912\2281456468.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btcusd_data_and_fed["350_movingaverage_per_FedAssets"] = pd.Series.rolling(btcusd_data_and_fed["BTC_per_FedAssets"], window=350, min_periods=1).mean()
C:\Users\gebel\AppData\Lo

# GET S&P DATA

In [10]:
# Billions of Dollars 
data_SP500 = fred.get_series('SP500')
data_SP500 = data_SP500 * 1000

data_SP500.dropna()
data_SP500 = data_SP500.sort_index()
# store current df with up-to-date values
data_SP500.to_csv('coindata/data_SP500.csv', index=True)

print(data_SP500.head(1))
print(data_SP500.tail(1))

2013-06-17    1639040.0
dtype: float64
2023-06-16    4409590.0
dtype: float64


# Assets: Total Assets: Total Assets: Wednesday Level (RESPPANWW)

In [11]:
start_date='2014-04-21'

In [12]:
#Assets: Total Assets: Total Assets: Wednesday Level (RESPPANWW) Millions of Dollars
data_WALCL = fred.get_series('WALCL')

data_WALCL.dropna()
data_WALCL = data_WALCL.sort_index()
# store current df with up-to-date values
data_WALCL.to_csv('coindata/data_WALCL.csv', index=True)

print(data_WALCL.head(1))
print(data_WALCL.tail(1))

2002-12-18    719542.0
dtype: float64
2023-06-14    8388323.0
dtype: float64


# Overnight Reverse Repurchase Agreements Treasury Securities Sold by the Federal Reserve in the Temporary Open Market Operations (RRPONTSYD)

In [13]:
data_RRPONTSYD = fred.get_series('RRPONTSYD')
data_RRPONTSYD

2003-02-07       2.500
2003-02-10         NaN
2003-02-11         NaN
2003-02-12         NaN
2003-02-13         NaN
                ...   
2023-06-14    2109.105
2023-06-15    1992.140
2023-06-16    2011.556
2023-06-19         NaN
2023-06-20    1989.489
Length: 5313, dtype: float64

In [14]:
#Assets: Total Assets: Total Assets: Wednesday Level (RESPPANWW)
# Billions of U.S. Dollars
# data_FRED_RRPONTSYD.values = data_FRED_RRPONTSYD.values * 1000 
data_RRPONTSYD = data_RRPONTSYD * 1000 
data_RRPONTSYD.dropna()
data_RRPONTSYD = data_RRPONTSYD.sort_index()
# store current df with up-to-date values
data_RRPONTSYD.to_csv('coindata/data_RRPONTSYD.csv', index=True)

print(data_RRPONTSYD.head(1))
print(data_RRPONTSYD.tail(1))

2003-02-07    2500.0
dtype: float64
2023-06-20    1989489.0
dtype: float64


# Deposits with Federal Reserve Banks, other than Reserve Balances: U.S. Treasury, General Account (WTREGEN)

In [15]:
# Billions of Dollars 
data_FRED_WTREGEN = fred.get_series('WTREGEN')
data_FRED_WTREGEN = data_FRED_WTREGEN * 1000

data_FRED_WTREGEN.dropna()
data_FRED_WTREGEN = data_FRED_WTREGEN.sort_index()
# store current df with up-to-date values
data_FRED_WTREGEN.to_csv('coindata/data_FRED_WTREGEN.csv', index=True)

print(data_FRED_WTREGEN.head(1))
print(data_FRED_WTREGEN.tail(1))

1986-01-08    4140.0
dtype: float64
2023-06-14    102118.0
dtype: float64


# net liquidity vs s&p500 weekly - Diagram

In [16]:
datasource_fred_total_assets = "data_WALCL.csv"
fred_total_assets = pd.read_csv("coindata/{}".format(
    datasource_fred_total_assets.replace("/", " ")), index_col=0)
fred_total_assets.index = pd.to_datetime(fred_total_assets.index)

most_recent_stored_fred_rrpontsyd_date = fred_total_assets.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")
todays_date = datetime.date.today() - timedelta(days=1)
todays_date = todays_date.strftime("%Y-%m-%d")
fred_total_assets

,0
2002-12-18,719542.0
2002-12-25,732059.0
2003-01-01,730994.0
2003-01-08,723762.0
2003-01-15,720074.0
...,...
2023-05-17,8456760.0
2023-05-24,8436255.0
2023-05-31,8385854.0
2023-06-07,8389325.0


In [17]:
datasource_FRED_WTREGEN = "data_FRED_WTREGEN.csv"
FRED_WTREGEN_data = pd.read_csv("coindata/{}".format(
    datasource_FRED_WTREGEN.replace("/", " ")), index_col=0)
FRED_WTREGEN_data.index = pd.to_datetime(FRED_WTREGEN_data.index)

most_recent_stored_FRED_WTREGEN_date = FRED_WTREGEN_data.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")
FRED_WTREGEN_data

,0
1986-01-08,4140.0
1986-01-15,5032.0
1986-01-22,13856.0
1986-01-29,17487.0
1986-02-05,12475.0
...,...
2023-05-17,116220.0
2023-05-24,61952.0
2023-05-31,48954.0
2023-06-07,44756.0


In [18]:
datasource_FRED_RRPONTSYD = "data_RRPONTSYD.csv"
FRED_RRPONTSYD_data = pd.read_csv("coindata/{}".format(
    datasource_FRED_RRPONTSYD.replace("/", " ")), index_col=0)
FRED_RRPONTSYD_data.index = pd.to_datetime(FRED_RRPONTSYD_data.index)

most_recent_stored_FRED_RRPONTSYD_date = FRED_RRPONTSYD_data.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")
FRED_RRPONTSYD_data

,0
2003-02-07,2500.0
2003-02-10,NaN
2003-02-11,NaN
2003-02-12,NaN
2003-02-13,NaN
...,...
2023-06-14,2109105.0
2023-06-15,1992140.0
2023-06-16,2011556.0
2023-06-19,NaN


In [19]:
type( FRED_RRPONTSYD_data.iloc[0].name )

pandas._libs.tslibs.timestamps.Timestamp

# filter for datetime to allign all fred data sources!

In [20]:
merged_FRED_RRPONTSYD_data = pd.merge(FRED_WTREGEN_data,FRED_RRPONTSYD_data, how='left', left_index=True, right_index=True)
# merged.isnull().sum()
del merged_FRED_RRPONTSYD_data["0_x"]
merged_FRED_RRPONTSYD_data.columns=["0"]

In [21]:
netLiquidity = fred_total_assets-merged_FRED_RRPONTSYD_data-FRED_WTREGEN_data

In [22]:
print(len(merged_FRED_RRPONTSYD_data))
print(len(fred_total_assets))
print(len(FRED_WTREGEN_data))

1954
1070
1954


In [23]:
# df['dt'] = pd.to_datetime(df['dt'])
FRED_RRPONTSYD_data = FRED_RRPONTSYD_data[(FRED_RRPONTSYD_data.index > '2010-08-11 00:00:00')]
fred_total_assets = fred_total_assets[(fred_total_assets.index > '2010-08-11 00:00:00')]
FRED_WTREGEN_data = FRED_WTREGEN_data[(FRED_WTREGEN_data.index > '2010-08-11 00:00:00')]

fred_total_assets.head()
fred_total_assets.columns = ['Value_fred_total_assets']
len(fred_total_assets)

FRED_WTREGEN_data.head()
FRED_WTREGEN_data.columns = ['Value_FRED_WTREGEN_data']
len(FRED_WTREGEN_data)

FRED_RRPONTSYD_data.head()
FRED_RRPONTSYD_data.columns = ['Value_FRED_RRPONTSYD_data']
len(FRED_RRPONTSYD_data)

# bring dfs together and forwardfill all!
frames = [FRED_RRPONTSYD_data, FRED_WTREGEN_data, fred_total_assets]
result = pd.concat(frames)

In [24]:
# result["netLIQ"] = result['Value_FRED_RRPONTSYD_data'] - result['Value_FRED_WTREGEN_data'] - result['Value_fred_total_assets'] 

In [25]:
# result["netLIQ"] = result['Value_FRED_RRPONTSYD_data'] - result['Value_FRED_WTREGEN_data'] - result['Value_fred_total_assets'] 

In [26]:
data_SP500

2013-06-17    1639040.0
2013-06-18    1651810.0
2013-06-19    1628930.0
2013-06-20    1588190.0
2013-06-21    1592430.0
                ...    
2023-06-12    4338930.0
2023-06-13    4369010.0
2023-06-14    4372590.0
2023-06-15    4425840.0
2023-06-16    4409590.0
Length: 2610, dtype: float64

In [27]:
# data_SP500_2weeksback = data_SP500.shift(-2,"W")
# data_SP500_2weeksback

In [28]:
data_SP500_1weekback = data_SP500.shift(-1, "W")
data_SP500_1weekback

2013-06-16    1639040.0
2013-06-16    1651810.0
2013-06-16    1628930.0
2013-06-16    1588190.0
2013-06-16    1592430.0
                ...    
2023-06-11    4338930.0
2023-06-11    4369010.0
2023-06-11    4372590.0
2023-06-11    4425840.0
2023-06-11    4409590.0
Length: 2610, dtype: float64

In [29]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

In [30]:
netLiquidity = netLiquidity[(netLiquidity.index > '2012-11-18 00:00:00')]
netLiquidity = netLiquidity.squeeze()
netLiquidity = netLiquidity.dropna()
netLiquidity

2013-04-03   3139168.00
2013-08-07   3522254.00
2013-08-14   3580992.00
2013-09-25   3682783.00
2013-10-02   3706724.00
                ...    
2023-05-17   6126864.00
2023-05-24   6123594.00
2023-05-31   6082041.00
2023-06-07   6183013.00
2023-06-14   6177100.00
Name: 0, Length: 503, dtype: float64

In [31]:
data_SP500_1weekback = data_SP500_1weekback[(data_SP500_1weekback.index > '2013-08-12 00:00:00')]
# data_SP500_1weekback = data_SP500_1weekback.index.drop_duplicates()
data_SP500_1weekback = data_SP500_1weekback[~data_SP500_1weekback.index.duplicated(keep='first')]
data_SP500_1weekback = data_SP500_1weekback.dropna()
data_SP500_1weekback

2013-08-18   1646060.00
2013-08-25   1656780.00
2013-09-08   1671710.00
2013-09-15   1697600.00
2013-09-22   1701840.00
                ...    
2023-05-07   4138120.00
2023-05-14   4136280.00
2023-05-21   4192630.00
2023-06-04   4273790.00
2023-06-11   4338930.00
Length: 463, dtype: float64

In [32]:
data_SP500_2weeksback = data_SP500_1weekback.shift(-1, "W")
data_SP500_2weeksback

2013-08-11   1646060.00
2013-08-18   1656780.00
2013-09-01   1671710.00
2013-09-08   1697600.00
2013-09-15   1701840.00
                ...    
2023-04-30   4138120.00
2023-05-07   4136280.00
2023-05-14   4192630.00
2023-05-28   4273790.00
2023-06-04   4338930.00
Length: 463, dtype: float64

In [33]:
dfdiffsp500_netliq = pd.concat([netLiquidity, data_SP500_1weekback], axis=1, keys=('netLiquidity','data_SP500_1weekback'), join='outer').ffill(axis = 0).dropna()
dfdiffsp500_netliq["netLiquidity"]  = dfdiffsp500_netliq["netLiquidity"] / 1.1 - 1625

dfdiffsp500_netliq["diff"] = dfdiffsp500_netliq["data_SP500_1weekback"]  - dfdiffsp500_netliq["netLiquidity"]  

In [42]:
from plotly.subplots import make_subplots


# # # start - plot fed net liquidity! # # #

fig_net_liq = make_subplots(specs=[[{"secondary_y": True}]])

fig_net_liq.add_trace(
    go.Scatter(
        x=dfdiffsp500_netliq.index,
        y=dfdiffsp500_netliq["diff"],
        name="diffsp500_netliq",
        mode='lines',
        marker=dict(
            # size=16,
            color="gold",  # set color equal to a variable
            # colorscale='Viridis', # one of plotly colorscales
            # showscale=True
        ), 
    ), 
    secondary_y=True,
)


fig_net_liq.add_trace(
    go.Scatter(
        x=btcusd_data.index,
        y=btcusd_data['Last'],
        name="BTC",
        mode='lines',
        marker=dict(
            # size=16,
            color="red",  # set color equal to a variable
            # colorscale='Viridis', # one of plotly colorscales
            # showscale=True
        ), 
    ), 
    secondary_y=True,
)

fig_net_liq.add_trace(
    go.Scatter(
        x=data_SP500_2weeksback.index,
        y=data_SP500_2weeksback.values,
        name="SP500 - 2 weeks ago",
        mode='lines',
        marker=dict(
            # size=16,
            color="blue",  # set color equal to a variable
            # colorscale='Viridis', # one of plotly colorscales
            # showscale=True
        ), 
    ), 
    secondary_y=True,
)


fig_net_liq.add_trace(
    go.Scatter(
        x=data_SP500_1weekback.index,
        y=data_SP500_1weekback.values,
        name="SP500 - 1 week ago",
        mode='lines',
        marker=dict(
            # size=16,
            color="blue",  # set color equal to a variable
            # colorscale='Viridis', # one of plotly colorscales
            # showscale=True
        ), 
    ), 
    secondary_y=True,
)

fig_net_liq.add_trace(go.Scatter(x=fred_total_assets.index, y=fred_total_assets["0"],
                    mode='lines',
                    name='fred_total_assets',
                    marker=dict(
                        color="black"
                        ),
                    )
                )

fig_net_liq.update_layout(yaxis=dict(domain=[0, 0.7]) )

fig_net_liq.add_trace(go.Scatter(x=netLiquidity.index, y=netLiquidity.values, stackgroup='one', #fill='tonexty',
                    mode='lines',
                    name='netLiquidity',
                    marker=dict(
                        color="green"
                        )
                    )
                )

fig_net_liq.add_trace(go.Scatter(x=merged_FRED_RRPONTSYD_data.index, y=merged_FRED_RRPONTSYD_data["0"], stackgroup='one', #fill='tonexty',
                    mode='lines',
                    name='merged_FRED_RRPONTSYD_data',
                    marker=dict(
                        color="yellow"
                        )
                    )
                )

fig_net_liq.add_trace(go.Scatter(x=FRED_WTREGEN_data.index, y=FRED_WTREGEN_data["0"], stackgroup='one', #fill='tonexty',
                    mode='lines',
                    name='FRED_WTREGEN_data',
                    marker=dict(
                        color="red"
                        ),
                    )
                )

fig.add_shape(type='line',
                x0=FRED_WTREGEN_data.index[0],
                y0=1250000,
                x1=FRED_WTREGEN_data.index[-1],
                y1=1250000,
                line=dict(color='Red',),
                xref='x',
                yref='y'
)
fig.add_shape(type='line',
                x0=FRED_WTREGEN_data.index[0],
                y0=300000,
                x1=FRED_WTREGEN_data.index[-1],
                y1=300000,
                line=dict(color='Red',),
                xref='x',
                yref='y'
)
                
fig_net_liq.update_layout(
    title="Fed net liquidity",
    autosize=False,
    width=int(1400/1),
    height=int(800/1),
)
fig_net_liq.update_yaxes(title_text="BTC & SP500", secondary_y=True)


In [35]:
aaaaasdasd

NameError: name 'aaaaasdasd' is not defined

In [ ]:
import plotly.express as px
df = px.data.gapminder()
fig = px.area(df, x="year", y="pop", color="continent", line_group="country")
fig.show()

In [ ]:
#Assets: Total Assets: Total Assets: Wednesday Level (RESPPANWW)
fed_assets_quandl_key = "FED/RESPPA_N_WW"

fed_assets_data = pd.read_csv("coindata/{}".format(
    fed_assets_quandl_key.replace("/", " ")), index_col=0)
fed_assets_data.index = pd.to_datetime(fed_assets_data.index)

In [ ]:
fed_assets_data.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")

'2022-12-14'

In [ ]:
from datetime import datetime, timedelta
import datetime
most_recent_stored_fed_assets_date = fed_assets_data.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")

todays_date = datetime.date.today() - timedelta(days=7)
todays_date = todays_date.strftime("%Y-%m-%d")


In [ ]:
most_recent_stored_fed_assets_date

'2022-12-14'

In [ ]:
todays_date

'2022-12-10'

In [ ]:
fig = go.Figure(
    data=go.Scatter(
        x=btcusd_data_and_fed.index,
        y=btcusd_data_and_fed['BTC_per_FedAssets'],
        name="BTC/FED",
        mode='lines',
        marker=dict(
            # size=16,
            color="red",  # set color equal to a variable
            # colorscale='Viridis', # one of plotly colorscales
            # showscale=True
        )
)
)

fig.add_trace(go.Scatter(x=btcusd_data_and_fed.index, y=btcusd_data_and_fed["350_movingaverage_per_FedAssets"],
                    mode='lines',
                    name='350_movingaverage_per_FedAssets',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="gold"
                        ),
                    )
                )

fig.add_trace(go.Scatter(x=btcusd_data_and_fed.index, y=btcusd_data_and_fed["350_movingaverage_per_FedAssets"]*2,
                    mode='lines',
                    name='2*350_movingaverage_per_FedAssets',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="red"
                        ),
                    )
                )

fig.add_trace(go.Scatter(x=btcusd_data.index, y=btcusd_data_and_fed["111_movingaverage_per_FedAssets"],
                    mode='lines',
                    name='111_movingaverage_per_FedAssets',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="green"
                        )
                    )
                )

# btcusd_data["111_movingaverage"]
fig.update_yaxes(type="log") #, range=[0,5]

fig.update_layout(
    # title="Plot Title",
    autosize=False,
    width=int(1400/1.3),
    height=int(800/1.3),
)

fig

In [ ]:
btcusd_data = btcusd_data.dropna()
btcusd_data["350_movingaverage"] = pd.Series.rolling(btcusd_data["Last"], window=350, min_periods=1).mean()
btcusd_data["111_movingaverage"] = pd.Series.rolling(btcusd_data["Last"], window=111, min_periods=1).mean()
btcusd_data

,High,Low,Mid,Last,Bid,Ask,Volume,First,350_movingaverage,111_movingaverage
Date,,,,,,,,,,
2014-04-16,547.00,495.00,537.50,538.00,537.00,538.00,29633.36,505.00,538.00,538.00
2014-04-17,538.50,486.10,507.02,508.00,506.04,508.00,20709.78,538.00,523.00,523.00
2014-04-18,509.00,474.25,483.77,482.75,482.75,484.79,10458.05,508.00,509.58,509.58
2014-04-19,513.99,473.83,505.01,507.50,502.53,507.49,8963.62,482.75,509.06,509.06
2014-04-20,518.00,492.20,500.75,501.44,500.07,501.42,4921.59,507.50,507.54,507.54
...,...,...,...,...,...,...,...,...,...,...
2022-12-12,17237.00,16879.00,17218.50,17219.00,17218.00,17219.00,2411.69,17104.00,29040.88,18916.04
2022-12-13,18000.00,17102.00,17788.50,17789.00,17788.00,17789.00,5961.75,17219.00,28955.34,18882.41
2022-12-14,18391.00,17678.00,17831.50,17832.00,17831.00,17832.00,3079.72,17789.00,28873.72,18848.38


In [ ]:
fig = go.Figure(
    data=go.Scatter(
        x=btcusd_data.index,
        y=btcusd_data['Last'],
        mode='lines',
        marker=dict(
            # size=16,
            color="black",  # set color equal to a variable
            # colorscale='Viridis', # one of plotly colorscales
            # showscale=True
        )
    )
)

fig.add_trace(go.Scatter(x=btcusd_data.index, y=btcusd_data["350_movingaverage"]*2,
                    mode='lines',
                    name='350_movingaverage',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="red"
                        ),
                    )
                )

fig.add_trace(go.Scatter(x=btcusd_data.index, y=btcusd_data["111_movingaverage"],
                    mode='lines',
                    name='111_movingaverage',
                    marker=dict(
                        # size=[40, 60, 80, 100],
                        color="green"
                        )
                    )
                )

# btcusd_data["111_movingaverage"]
fig.update_yaxes(type="log") #, range=[0,5]

fig.update_layout(
    # title="Plot Title",
    autosize=False,
    width=int(1400/1.3),
    height=int(800/1.3),
)

fig